In [ ]:
# Group number 1
# Assignment 7: Development of optimal smoothing to increase the estimation accuracy
# Team members: Stepan Perminov, Polina Ponomareva, Kirill Shcherbakov, Daniil Svirskiy

# Library importing
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

# Size of the trajectory
n = 200

# Time step
T = 1

# Variances
sigma_a2 = 0.2**2
sigma_et2 = 20**2

# Errors initialization
Error_filt = np.zeros((2,n,500))
Error_smoothed = np.zeros((2,n,500))

for k in range(500):
    # Generation of normally distributed random noises with zero mathematical expectation and corresponding variances
    a = np.random.normal(0, np.sqrt(sigma_a2), n-1)
    et = np.random.normal(0, np.sqrt(sigma_et2), n)

    # Matrixes
    F = np.array([[1, T],[0, 1]])
    G = np.array([[(T**2)/2.0],[T]])
    H = np.array([1, 0])

    # Initial X
    X = np.array([[5],[1]])
    X_var = X

    # Generation of true trajectory X
    for i in range(1,len(a)+1):
        X = np.hstack((X, F.dot(X_var) + G*a[i-1]))
        X_var = F.dot(X_var) + G*a[i-1]

    # Generation of measurements z
    z = H.dot(X) + et

    # Covariance matrixes
    Q = G*G.T*sigma_a2
    R = sigma_et2

    # Initialization of matrixes
    P_1 = np.zeros((2,2,n))
    X_1 = np.zeros((2,n))
    P_ = np.zeros((2,2,n))
    X_ = np.zeros((2,n))
    HT = H.reshape(2,1)
    K = np.zeros((2,n))

    # Initial P for filtering
    P_[:,:,0] = [[10000, 0],[0, 10000]]

    # Initial X_filt for filtering
    X_[:,0] = [2, 0]

    # Kalman filtering
    for i in range(1,n):
        P_1[:,:,i] = (F.dot(P_[:,:,i-1])).dot(F.T) + Q

        X_1[:,i] = F.dot(X_[:,i-1].reshape(2,1)).reshape(2)
        K[:,i] = ((P_1[:,:,i].dot(HT))/((H.dot(P_1[:,:,i])).dot(HT) + R)).reshape(2)

        X_[:,i] = X_1[:,i] + K[:,i]*(z[i] - H.dot(X_1[:,i]))
        P_[:,:,i] = P_1[:,:,i] - (K[:,i].reshape(2,1)*H).dot(P_1[:,:,i])

    K = np.delete(K, 0, axis = 1)
    
    # Backward smoothing
    X_new = np.zeros((2,n))
    X_new[:,-1] = X_[:,-1]
    P_new = np.zeros((2,2,n))
    P_new[:,:,-1] = P_[:,:,-1]

    for i in reversed(range(n-1)):
        Ai = P_[:,:,i].dot(F.T).dot(inv(P_1[:,:,i+1]))
        X_new[:,i] = X_[:,i] + Ai.dot(X_new[:,i+1] - F.dot(X_[:,i]))
        P_new[:,:,i] = P_[:,:,i] + Ai.dot(P_new[:,:,i+1] - P_1[:,:,i+1]).dot(Ai.T)
        
    
    Error_filt[:,:,k] = (X - X_)**2
    Error_smoothed[:,:,k] = (X - X_new)**2

In [ ]:
# Final errors
Final_err_filt = np.sqrt(np.sum(Error_filt, axis = 2)/(500.0-1.0))
Final_err_smoothed = np.sqrt(np.sum(Error_smoothed, axis = 2)/(500.0-1.0))

In [ ]:
# Compare of true smoothing and filtration errors of estimation of coordinates
plt.plot(Final_err_filt[0,3:], linewidth = 6, label = "True errors of filtered estimates", color = "y")
plt.plot(Final_err_smoothed[0,3:], linewidth = 6, label = "True errors of smoothed estimates", color = "g")
plt.legend(fontsize = 70)
plt.rcParams["figure.figsize"] = (50,30)
plt.tick_params(labelsize = 60)
plt.grid()
plt.xlabel("Points", fontsize = 70)
plt.ylabel("Values of processes", fontsize = 70)
plt.title("Compare of true smoothing and filtration errors of estimation of coordinates\n", fontsize = 70)
plt.show()

In [ ]:
# Compare of true errors of coordinates and errors provided by smoothing algorithm
plt.plot(Final_err_smoothed[0,3:], linewidth = 6, label = "True errors of smoothed estimates", color = "y")
plt.plot(np.sqrt(P_new[0,0,3:]), linewidth = 6, label = "Errors provided by smoothing algorithm", color = "g")
plt.legend(fontsize = 70)
plt.rcParams["figure.figsize"] = (50,30)
plt.tick_params(labelsize = 60)
plt.grid()
plt.xlabel("Points", fontsize = 70)
plt.ylabel("Values of processes", fontsize = 70)
plt.title("Compare of true errors of coordinates and errors provided by smoothing algorithm\n", fontsize = 70)
plt.show()

In [ ]:
# Compare of true smoothing and filtration errors of estimation of velocities
plt.plot(Final_err_filt[1,3:], linewidth = 6, label = "True errors of filtered estimates", color = "y")
plt.plot(Final_err_smoothed[1,3:], linewidth = 6, label = "True errors of smoothed estimates", color = "g")
plt.legend(fontsize = 70)
plt.rcParams["figure.figsize"] = (50,30)
plt.tick_params(labelsize = 60)
plt.grid()
plt.xlabel("Points", fontsize = 70)
plt.ylabel("Values of processes", fontsize = 70)
plt.title("Compare of true smoothing and filtration errors of estimation of velocities\n", fontsize = 70)
plt.show()

In [ ]:
# Compare of true errors of velocities and errors provided by smoothing algorithm
plt.plot(Final_err_smoothed[1,3:], linewidth = 6, label = "True errors of smoothed estimates", color = "y")
plt.plot(np.sqrt(P_new[1,1,3:]), linewidth = 6, label = "Errors provided by smoothing algorithm", color = "g")
plt.legend(fontsize = 70)
plt.rcParams["figure.figsize"] = (50,30)
plt.tick_params(labelsize = 60)
plt.grid()
plt.xlabel("Points", fontsize = 70)
plt.ylabel("Values of processes", fontsize = 70)
plt.title("Compare of true errors of velocities and errors provided by smoothing algorithm\n", fontsize = 70)
plt.show()

In [ ]:
# Plot of comparison of coordinate trajectories
points = range(1,n+1)
plt.plot(points, z, linewidth = 6, label = "Measurements", color = "y")
plt.plot(points, X[0,:], linewidth = 6, label = "True coordinates", color = "g")
plt.plot(points, X_[0,:], linewidth = 6, label = "Kalman filtered estimates", color = "c")
plt.plot(points, X_new[0,:], linewidth = 6, label = "Backward smoothed estimates", color = "b")
plt.legend(fontsize = 80)
plt.rcParams["figure.figsize"] = (50,30)
plt.tick_params(labelsize = 60)
plt.grid()
plt.xlabel("Points", fontsize = 80)
plt.ylabel("Values of processes", fontsize = 80)
plt.title("Comparison of coordinates\n", fontsize = 80)
plt.show()

In [ ]:
# Plot of comparison of velocity trajectories
plt.plot(points, X[1,:], linewidth = 6, label = "True velocity", color = "g")
plt.plot(points, X_[1,:], linewidth = 6, label = "Kalman filtered estimates", color = "c")
plt.plot(points, X_new[1,:], linewidth = 6, label = "Backward smoothed estimates", color = "b")
plt.legend(fontsize = 80)
plt.rcParams["figure.figsize"] = (50,30)
plt.tick_params(labelsize = 60)
plt.grid()
plt.xlabel("Points", fontsize = 80)
plt.ylabel("Values of processes", fontsize = 80)
plt.title("Comparison of velocities\n", fontsize = 80)
plt.show()